In [1]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 5.1 MB/s eta 0:00:00


In [2]:
!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 13.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 43.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 11.0.0
    Uninstalling pyarrow-11.0.0:
      Successfully uninstalled pyarrow-11.0.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.12.2
    Uninstalling fsspec-2023.12.2:
      Successfully uninstalled fsspec-2023.12.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-c

In [3]:
import pandas as pd

from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
import torch

# Check for CUDA-enabled devices
if torch.cuda.is_available():
    num_devices = torch.cuda.device_count()
    print(f"Number of CUDA devices available: {num_devices}")
    print("CUDA device names:")
    for i in range(num_devices):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available.")
    
device = torch.device("cuda")



Number of CUDA devices available: 1
CUDA device names:
Device 0: Tesla P100-PCIE-16GB


In [4]:
# device = torch.device(0) 

In [5]:
import re

def remove_before_x(response):
    response=response.replace("equation:","")
    response=response.replace(" ","")
    response=response.replace("</s>","")
    pattern = re.compile(r'.*?(x.*)')
    result = re.sub(pattern, r'\1', response)
    return result

from difflib import SequenceMatcher

def reward_function(str1, str2):
    # Using SequenceMatcher to get similarity ratio
    similarity_ratio = SequenceMatcher(None, str1, str2).ratio()
    
    # Adjusting reward based on the difference in length
    length_penalty = min(len(str1), len(str2)) / max(len(str1), len(str2))
    
    return similarity_ratio * length_penalty

In [6]:
# str1 = "x-10=5"
# str2 = "x-10=52"

# def reward1(str1,str2):
#     count = 0
#     if len(str1)>len(str2):
#         for i in range(len(str2)):
#             if str1[i]==str2[i]:
#                 count += 1
#         return count/len(str2)
#     elif len(str1)==len(str2):
#         for i in range(len(str2)):
#             if str1[i]==str2[i]:
#                 count += 1
#         return count/len(str2)
#     elif len(str1)<len(str2):
#         for i in range(len(str1)):
#             if str1[i]==str2[i]:
#                 count += 1
#         return count/len(str1)
    
# def reward2(str1,str2):
#     len1= (len(str1))
#     len2= (len(str2))
#     if len1==len2:
#         return 1
#     elif len1>len2:
#         return len2/len1
#     elif len2>len1:
#         return len1/len2

In [7]:

# import random
# import wandb
# import time
from tqdm import tqdm
from random import choices
import numpy as np
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, create_reference_model, AutoModelForSeq2SeqLMWithValueHead
                

2024-02-09 11:20:32.625198: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-09 11:20:32.625314: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-09 11:20:32.759680: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
# import torch
# device = torch.device("cpu")
# # print(torch.cuda.get_device_name(0))

In [9]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

loaded_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained("sammanamgain/T5_GOOGLE_BASE")
loaded_tokenizer = AutoTokenizer.from_pretrained("sammanamgain/T5_GOOGLE_BASE")

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [48]:
config = PPOConfig(
    model_name= 'T5ForConditionalGeneration',
    batch_size=32,
    learning_rate=2e-5,
    remove_unused_columns=False, log_with="wandb",
    gradient_accumulation_steps=4,
    ratio_threshold = 55
)

np.random.seed(42)

In [49]:
## from transformers import AutoTokenizer
t5_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained("sammanamgain/T5_GOOGLE_BASE")
# t5_model.to(device1)
t5_model_ref = create_reference_model(t5_model)  
# t5_model_ref.to(device2)
t5_tokenizer = loaded_tokenizer

In [50]:
generation_kwargs = {
    "min_length": 1,
    "num_beams": 5,  # lookahead parameter
    "no_repeat_ngram_size": 5,  # prescence penalty
    "do_sample": True,
    "pad_token_id": t5_tokenizer.pad_token_id,
    "max_length": 30,
    "eos_token_id": t5_tokenizer.eos_token_id,
}

In [51]:
from datasets import DatasetDict, Dataset

# Load your CSV file into a pandas DataFrame
import pandas as pd

# Replace 'your_dataset.csv' with the actual path to your CSV file
df = pd.read_csv('/kaggle/input/train-t5-base/train_set.csv')

# Define a function to process each row and create a new structure
def process_row(row):
    return {
        'Question': row['Question'],
        'Equation': row['Equation'],
    }

# Apply the processing function to each row
processed_data = df.apply(process_row, axis=1).tolist()

# Split the data into train and test sets (you can adjust the ratio)
train_data = processed_data[::]
test_data = processed_data[0:1]

# Create DatasetDict
data_dict = DatasetDict({
    'train': Dataset.from_dict({'Question': [item['Question'] for item in train_data], 'Equation': [item['Equation'] for item in train_data]}),
    'test': Dataset.from_dict({'Question': [item['Question'] for item in test_data], 'Equation': [item['Equation'] for item in test_data]}),
})

In [52]:
def get_reward(text,response,df):
    reward = []
    for i in range(len(response)):
        print(text[i])
        index = np.where(df == text[i])
        # print(df.iloc[index[0][0],0])
        # print(df.iloc[index[0][0],1])
        # print(response[i])
        # print(text[i])
        row_index =int(index[0][0])
        original = remove_before_x(df.iloc[row_index,1])
        predicted = remove_before_x(response[i])
#         r1 = reward1(df.iloc[row_index,1],response[i])
#         r2= reward2(df.iloc[row_index,1],response[i])
        total_r = reward_function(original,predicted)
        reward.append(total_r)
        print('--------------------------------')
        print('Original : ' + original)
        print('')
        print('Reward is : ')
        print(total_r)
        print('Predicted: '+ predicted)
        print('--------------------------------')
        print('')
    return reward

In [53]:
dataset = data_dict.map(
    lambda x: {"input_ids": t5_tokenizer.encode('Math word question to equation: ' + x["Question"], return_tensors="pt")},
    batched=False,
)

Map:   0%|          | 0/17853 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [54]:
dataset.set_format("pytorch")

In [55]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

In [56]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Question', 'Equation', 'input_ids'],
        num_rows: 17853
    })
    test: Dataset({
        features: ['Question', 'Equation', 'input_ids'],
        num_rows: 1
    })
})


In [57]:
# # Clear memory on device1
# with torch.cuda.device(device1):
#     torch.cuda.empty_cache()
#     # Collect garbage to release more memory
#     torch.cuda.synchronize()
#     for obj in list(globals().values()):
#         if torch.is_tensor(obj):
#             del obj

# # Clear memory on device2
# with torch.cuda.device(device2):
#     torch.cuda.empty_cache()
#     # Collect garbage to release more memory
#     torch.cuda.synchronize()
#     for obj in list(globals().values()):
#         if torch.is_tensor(obj):
#             del obj

In [58]:
ppo_trainer = PPOTrainer(
    config, t5_model, t5_model_ref, t5_tokenizer, dataset['train'], data_collator=collator
)

env/reward,█▄▁▂▃█▆▆▅██
env/reward_mean,█▄▁▂▃█▆▆▅██
env/reward_std,▁▆█▇▅▄▃▃▄▂▂
objective/entropy,▁▂▄█▅▁▅▅▄▄▄
objective/kl,▁▄▆▆▄▆▂▇▃█▅
objective/kl_coef,█▇▇▆▅▄▄▃▂▂▁
ppo/learning_rate,▁▁▁▁▁▁▁▁▁▁▁
ppo/loss/policy,▁▂█▃▅▁▃▃▃▂▂
ppo/loss/total,▁▂█▃▅▁▃▃▃▂▂
ppo/loss/value,▂▃▃▃▂▁▁█▂▃▂
ppo/mean_non_score_reward,█▅▃▃▅▃▆▁▅▁▄


In [59]:
# def get_reward(response):
#     scores = []
#     # note function_to_apply='none' gives me logits which can be negative
#     x = 0
#     for result in response:
#         if result==df['text'][x]:
#             scores.append(float(1))
#         else:
#             scores.append(float(0))
#     return scores


In [60]:
# from tqdm.auto import tqdm
# for epoch in tqdm(range(2)):
#     for batch in tqdm(ppo_trainer.dataloader):
#         game_data = dict()
#         #### prepend the generate question token
#         game_data["query"] = ['Math word question to equation: ' + b for b in batch["text"]]

#         #### get response from t5-model
#         input_tensors = [_.squeeze() for _ in batch["input_ids"]]
#         response_tensors = []
#         for query in input_tensors:
#             response = ppo_trainer.generate(query.squeeze(), **generation_kwargs)
#             response_tensors.append(response.squeeze())
        
#         game_data["response"] = [t5_tokenizer.decode(r.squeeze(), skip_special_tokens=True) for r in response_tensors]
        
#         # game_data["reward"] = get_reward(game_data["response"])
#         # rewards = game_data['reward']
#         # print(batch["text"],game_data['response'])
#         print(get_reward(batch["text"],game_data['response'],df))

In [61]:
# import torch

# # Perform operations to potentially allocate GPU memory

# # Clear GPU memory
# torch.cuda.empty_cache()


In [ ]:
from tqdm.auto import tqdm
for epoch in tqdm(range(5)):
    for batch in tqdm(ppo_trainer.dataloader):
        game_data = dict()
        #### prepend the generate question token
        game_data["query"] = ['Math word question to equation: ' + b for b in batch["Question"]]

        #### get response from t5-model
        input_tensors = [_.squeeze() for _ in batch["input_ids"]]
        response_tensors = []
        for query in input_tensors:
            response = ppo_trainer.generate(query.squeeze(), **generation_kwargs)
            response_tensors.append(response.squeeze())
        
        game_data["response"] = [t5_tokenizer.decode(r.squeeze(), skip_special_tokens=True) for r in response_tensors]
        game_data["reward"] = get_reward(batch["Question"],game_data["response"],df)
        rewards = game_data['reward']
        print(game_data['reward'])

        # Calculate the averages for each index
        # rewards = [1 * values[0] +  0.5 * values[1] for values in transposed_lists]
        rewards = [torch.tensor([_]) for _ in rewards]
        print(rewards)  # just to inspect :)
        stats = ppo_trainer.step(input_tensors, response_tensors, rewards)

        stats['env/reward'] = np.mean([r.cpu().numpy() for r in rewards])
        ppo_trainer.log_stats(stats, game_data, rewards)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/557 [00:00<?, ?it/s]

A box contains 45 red and green balls. If there are 5 more green balls than red balls, how many of each are there?
--------------------------------
Original : x+y=45,y=x+5

Reward is : 
1.0
Predicted: x+y=45,y=x+5
--------------------------------

The difference between the ages of Penelope and her son, Zack, is 34. In six years, Penelope will be four times as old as Zack’s age two years ago. How old are they now?
--------------------------------
Original : x-y=34,x+6=4(y-2)

Reward is : 
0.8450292397660819
Predicted: x-y=34,x+y+6=4(y-2)
--------------------------------

There are 7 crayons in the drawer . Mary took 3 crayons out of the drawer . How many crayons are there now ?
--------------------------------
Original : x=7-3

Reward is : 
0.8
Predicted: X=7-3
--------------------------------

Hope Primary School enrolls 13250 classes of first-grade freshmen, and each class enrolls 48 students. How many first-grade freshmen are enrolled in total?
--------------------------------
Origi

In [ ]:
t5_model.save_pretrained("RL_T5_3_epohcs")
t5_tokenizer.save_pretrained("RL_T5_3_epohcs")

In [ ]:
loaded_model_path = "two_var_t5"
t5_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained('/kaggle/input/rl_4_epoch/pytorch/rl-test-4-epoch/1')
t5_tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/rl_4_epoch/pytorch/rl-test-4-epoch/1')

In [ ]:
rl_model = AutoModelForSeq2SeqLM.from_pretrained('/kaggle/input/rl_5_epoch/pytorch/rl_5_epoch/1')
rl_tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/rl_5_epoch/pytorch/rl_5_epoch/1')

In [ ]:
input_ids = t5_tokenizer(
    "English to Math Equation: The age of Mary is twice the age of Steve. Their combined age is 66. Find Mary and Steve's ages.",
    return_tensors="pt"
).input_ids.to()
outputs = t5_model.generate(input_ids,max_length = 200)
print(t5_tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
input_ids = t5_tokenizer(
    "English to Math Equation: Ram bought 10 mangoes. He gave 2 mangoes to Shyam, 3 mangoes to Hari. How much mangoes does he have left?",
    return_tensors="pt"
).input_ids.to(device)
outputs = t5_model.generate(input_ids)
print(t5_tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
input_ids = t5_tokenizer(
    "English to Math Equation: Ukg sir gave 2 pens to saroj sir. He had 10 in total, how many does he have left?",
    return_tensors="pt"
).input_ids
outputs = t5_model.generate(input_ids)
print(t5_tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
input_ids = t5_tokenizer(
    "English to Math Equation: A baker buys 35 cakes and 28 pastries for $840. Afterwards, he buys 18 cakes and 24 pastries for $420. Find the cost of each cake and each pastry.",
    return_tensors="pt"
).input_ids
outputs = t5_model.generate(input_ids,max_length = 200)
print(t5_tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
input_ids = rl_tokenizer(
    "English to Math Equation: Erin's age is 3 times Warren's. In 4 years she will be twice as old as he will be. how old is each now?"
    ,return_tensors="pt"
).input_ids
outputs = rl_model.generate(input_ids, max_length = 30)
print(rl_tokenizer.decode(outputs[0], skip_special_tokens=True))